<a href="https://colab.research.google.com/github/farihahahaa/DiscoveryFall22/blob/main/InitDataScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import glob # package that will help us grab all the pdfs in a folder
from pathlib import Path
from tqdm import tqdm
import re
import operator
import math
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline

pd.set_option("display.max_columns", 330)
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore")

#!pip install pdfplumber -q
# import pdfplumber

In [2]:
!pip install tabula-py
# !pip install tabula-py[jpype]
import tabula

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 16.8 MB/s eta 0:00:00


In [3]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)
#filepath_research1
# research1 = pd.read_csv(filepath_research1, index_col=0)

#test_9_app = "../Downloading_LIHTC_Applications/2017_lihtc_9%/17-001.pdf"

Mounted at /content/gdrive


In [ ]:
test_9_app = "/content/gdrive/MyDrive/Terner/PDFs/17-003.pdf"
#Downloading_LIHTC_Applications/2017_lihtc_9%/17-024.pdf
#documentation here https://tabula-py.readthedocs.io/en/latest/tabula.html
tabula.convert_into(test_9_app, "test_9%.csv", output_format="csv", pages=[22,23])
#pg 22 and 23 is sources and use budget for this app
#tabula generates csvs, so read back in what it generated
tabula_test_output = pd.read_csv("test_9%.csv", header=1)
df = tabula_test_output.copy()
df.head()

,Unnamed: 0,TOTAL\rPROJECT\rCOST,RES. COST,COM'L. COST,TAX CREDIT\rEQUITY,1)Citibank,2)Los Angeles\rCounty\rCommunity\rDevelopment\rCommission,3)Deferred\rDeveloper Fee,4)LA CDC Fee\rWaiver,5)GP Equity,6),7),8),9),10),11),12),SUBTOTAL,70% PVC for\rNew\rConst/Rehab,30% PVC for\rAcquisition
0,LAND COST/ACQUISITION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1Land Cost or Value,"$1,600,000","$1,600,000",NaN,"$1,600,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$1,600,000",NaN,NaN
2,2Demolition,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Legal,"$30,000","$30,000",NaN,"$30,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$30,000",NaN,NaN
4,Land Lease Rent Prepayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#, 'TOTAL\rPROJECT\rCOST': 'TOTAL PROJECT COST', 'TAX CREDIT\rEQUITY': 'TAX CREDIT EQUITY', '2)Los Angeles\rCounty\rCommunity\rDevelopment\rCommission': '2)Los Angeles County Community Development Commission'

In [ ]:
df=df.rename(columns = {'Unnamed: 0':'LAND_COST/ACQUISITION'})
#df.columns = df.columns.str.extract(r'\\r', expand=False)
#df=df.rename(columns = {'Unnamed: 0':'LAND_COST/ACQUISITION'}) .replace(r'/[\r\n]', " ")
# def clean_column_names(columns):
#     return [re.sub(r'\r', '', col) for col in columns]
# df.columns = clean_column_names(df.columns)
df.columns = df.columns.str.replace('\r', '_')

# print("Original Column Names:", [repr(col) for col in df.iloc[0]])
# cleaned_columns = clean_column_names(df.iloc[0])
# print("Cleaned Column Names:", [repr(col) for col in cleaned_columns])
# df.columns = cleaned_columns
df = df.iloc[1:]
df.head(5)

,LAND_COST/ACQUISITION,TOTAL_PROJECT_COST,RES. COST,COM'L. COST,TAX CREDIT_EQUITY,1)Citibank,2)Los Angeles_County_Community_Development_Commission,3)Deferred_Developer Fee,4)LA CDC Fee_Waiver,5)GP Equity,6),7),8),9),10),11),12),SUBTOTAL,70% PVC for_New_Const/Rehab,30% PVC for_Acquisition
1,1Land Cost or Value,"$1,600,000","$1,600,000",NaN,"$1,600,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$1,600,000",NaN,NaN
2,2Demolition,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Legal,"$30,000","$30,000",NaN,"$30,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$30,000",NaN,NaN
4,Land Lease Rent Prepayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1Total Land Cost or Value,"$1,630,000","$1,630,000",NaN,"$1,630,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$1,630,000",NaN,NaN


In [ ]:
df['TOTAL_PROJECT_COST']

1      $1,600,000
2             NaN
3         $30,000
4             NaN
5      $1,630,000
          ...    
113           NaN
114           NaN
115           NaN
116           NaN
117           NaN
Name: TOTAL_PROJECT_COST, Length: 117, dtype: object

Following is code to use (uncomment lines to import file)

In [ ]:
# !pip install tabula-py
# import tabula
# from google.colab import drive
# drive.mount("/content/gdrive", force_remount=True)
# app = "/content/gdrive/MyDrive/Terner/PDFs/_______.pdf"
#tabula.convert_into(app, "test_9%.csv", output_format="csv", pages=[22,23])
#tabula_test_output = pd.read_csv("test_9%.csv", header=1)
#df = tabula_test_output.copy()

def import_and_clean_table(pdf):
  csv = tabula.convert_into(pdf, f"{pdf}.csv", output_format="csv", pages=[22,23])
  df = pd.read_csv(csv, header=1) #csv.get()
  df=df.rename(columns = {'Unnamed: 0':'LAND_COST/ACQUISITION'})
  def clean_column_names_r(columns):
      return [re.sub(r'\r', '', col) for col in columns]
  df.columns = clean_column_names_r(df.columns)
  df = df.iloc[1:]
  return df

df.head(5)

,LAND_COST/ACQUISITION,TOTAL_PROJECT_COST,RES. COST,COM'L. COST,TAX CREDIT_EQUITY,1)Citibank,2)Los Angeles_County_Community_Development_Commission,3)Deferred_Developer Fee,4)LA CDC Fee_Waiver,5)GP Equity,6),7),8),9),10),11),12),SUBTOTAL,70% PVC for_New_Const/Rehab,30% PVC for_Acquisition
1,1Land Cost or Value,"$1,600,000","$1,600,000",NaN,"$1,600,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$1,600,000",NaN,NaN
2,2Demolition,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Legal,"$30,000","$30,000",NaN,"$30,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$30,000",NaN,NaN
4,Land Lease Rent Prepayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1Total Land Cost or Value,"$1,630,000","$1,630,000",NaN,"$1,630,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$1,630,000",NaN,NaN


In [ ]:
import tabula
import pandas as pd

def extract_table_from_pdf(pdf_path, output_csv_path):
    # Extract tables from the PDF
    #tables = tabula.read_pdf(pdf_path, pages='22-23', multiple_tables=True)
    # csv = tabula.convert_into(pdf_path, f"{pdf_path}.csv", output_format="csv", pages=[22,23])
    # df = pd.read_csv(csv, header=1)

    # # Concatenate tables if more than one table is extracted
    # df = pd.concat(tables, ignore_index=True)

    tabula.convert_into(pdf_path, output_csv_path, output_format="csv", pages='22-23')

    # Read the CSV into a DataFrame
    df = pd.read_csv(output_csv_path)

    # Clean the column names
    df=df.rename(columns = {'Unnamed: 0':'LAND_COST/ACQUISITION'})
    #df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w\s]', '')
    df.columns = df.columns.str.replace('\r', '_')
    df = df.iloc[1:]

    # Save the DataFrame to a CSV file
    df.to_csv(output_csv_path, index=False)

    #csv = tabula.convert_into(pdf, f"{pdf}.csv", output_format="csv", pages=[22,23])
    #df = pd.read_csv(csv.get(), header=1)
    #df=df.rename(columns = {'Unnamed: 0':'LAND_COST/ACQUISITION'})
    # def clean_column_names_r(columns):
    #   return [re.sub(r'\r', '', col) for col in columns]
    # df.columns = clean_column_names_r(df.columns)
    df = df.iloc[1:]

    return df

# Usage
pdf_path = "/content/gdrive/MyDrive/Terner/PDFs/17-004.pdf"
output_csv_path = "/content/gdrive/MyDrive/Terner/CSVs/17-004.csv"
cleaned_df = extract_table_from_pdf(pdf_path, output_csv_path)

# Display the cleaned DataFrame
print(cleaned_df)


    IV. SOURCES AND USES BUDGET - SECTION 1: SOURCES AND USES BUDGET  \
2                                                1Land Cost or Value   
3                                                        2Demolition   
4                                                              Legal   
5                                         Land Lease Rent Prepayment   
6                                          1Total Land Cost or Value   
..                                                               ...   
114                                                              NaN   
115                                                              NaN   
116                                                              NaN   
117                                                              NaN   
118                                                              NaN   

    Permanent Sources  Unnamed: 2 Unnamed: 3  Unnamed: 4 Unnamed: 5  \
2          $4,793,931  $4,793,931        NaN  $4,793,931        

In [4]:
#drive.mount("/content/gdrive", force_remount=True)
app_24 = "/content/gdrive/MyDrive/Terner/PDFs/17-024.pdf"
#Downloading_LIHTC_Applications/2017_lihtc_9%/17-024.pdf
#documentation here https://tabula-py.readthedocs.io/en/latest/tabula.html
tabula.convert_into(app_24, "24app_9%.csv", output_format="csv", pages=[12])
#pg 22 and 23 is sources and use budget for this app
#tabula generates csvs, so read back in what it generated
tabula_test_output = pd.read_csv("24app_9%.csv")
#had to remove header=1 from inside pd.read_csv!
df24 = tabula_test_output.copy()
df24.head()

Jul 12, 2024 8:28:36 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Jul 12, 2024 8:28:36 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>



,Unnamed: 0,Name of Lender/Source,Term (months) Interest Rate,Unnamed: 3,Amount of Funds
0,1),Wells Fargo Bank Construction Loan,18 3.750%,NaN,"$10,923,382"
1,2),Sonoma County CDC - Fund for Housing,660 3.000%,NaN,"$2,273,440"
2,3),Federal Home Loan Bank-SF-AHP,660 zero,NaN,"$800,000"
3,4),Land Donation from Unrelated Party,NaN,NaN,"$1,108,316"
4,5),LP Equity available during construction,NaN,NaN,"$1,400,000"


In [9]:
!pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 51.2 MB/s eta 0:00:00


In [27]:
import pdfplumber
import pandas as pd

def extract_table_from_pdf(pdf_path, page_number, output_csv_path):
    """
    Extracts a table from a PDF and saves it as a CSV file.

    Parameters:
        pdf_path (str): Path to the PDF file.
        page_number (int): Page number to extract the table from.
        output_csv_path (str): Path to save the extracted table as a CSV file.
    """
    with pdfplumber.open(pdf_path) as pdf:
        # Get the specific page
        page = pdf.pages[page_number - 1]

        # Extract the table
        table = page.extract_table()

        if table:
            # Convert the table to a DataFrame
            df = pd.DataFrame(table[1:], columns=table[0])

            # Save the DataFrame to a CSV file
            df.to_csv(output_csv_path, index=False)
            print(f"Table extracted and saved to {output_csv_path}")
        else:
            print("No table found on the specified page.")

# # Example usage
# pdf_path = "path/to/your/file.pdf"
# page_number = 1  # Specify the page number where the table is located
# output_csv_path = "output.csv"


plu_24 = extract_table_from_pdf(app_24, 12, "24app_9%.csv")
plu_24 = pd.read_csv("24app_9%.csv")
plu_24.head()
plu_24 = plu_24.drop('Name of Lender/Source', axis=1)
plu_24 = plu_24.rename(columns={'Unnamed: 1': 'const_l1_name', 'Term (months)': 'const_l1_term', 'Interest Rate': 'const_l1_ir', 'Amount of Funds': 'const_l1_fund'})
plu_24_total = plu_24.iloc[12, 3]
plu_24_total
plu_24

Table extracted and saved to 24app_9%.csv


,const_l1_name,const_l1_term,const_l1_ir,const_l1_fund
0,Wells Fargo Bank Construction Loan,18.0,3.750%,"$10,923,382"
1,Sonoma County CDC - Fund for Housing,660.0,3.000%,"$2,273,440"
2,Federal Home Loan Bank-SF-AHP,660.0,zero,"$800,000"
3,Land Donation from Unrelated Party,NaN,NaN,"$1,108,316"
4,LP Equity available during construction,NaN,NaN,"$1,400,000"
5,GP equity,NaN,NaN,"$1,000"
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [33]:
import pdfplumber
import pandas as pd

def extract_table_from_pdf(pdf_path, page_number, output_csv_path):
    """
    Extracts a table from a PDF and saves it as a CSV file.

    Parameters:
        pdf_path (str): Path to the PDF file.
        page_number (int): Page number to extract the table from.
        output_csv_path (str): Path to save the extracted table as a CSV file.
    """
    with pdfplumber.open(pdf_path) as pdf:
        # Get the specific page
        page = pdf.pages[page_number - 1]

        # Extract the table
        table = page.extract_table()

        if table:
            # Convert the table to a DataFrame
            df = pd.DataFrame(table[1:], columns=table[0])

            # Save the DataFrame to a CSV file
            df.to_csv(output_csv_path, index=False)
            print(f"Table extracted and saved to {output_csv_path}")
        else:
            print("No table found on the specified page.")

# # Example usage
# pdf_path = "path/to/your/file.pdf"
# page_number = 1  # Specify the page number where the table is located
# output_csv_path = "output.csv"


plu_24 = extract_table_from_pdf(app_24, 14, "24app_permfin.csv")
pf_24 = pd.read_csv("24app_permfin.csv")
pf_24.head()
pf_24 = pf_24.drop('Name of Lender/Source', axis=1)
pf_24 = pf_24.rename(columns={'Unnamed: 1': 'perm_l1_name', 'Term\n(months)': 'perm_l1_term', 'Interest\nRate': 'const_l1_ir',
'Residual\nReceipts /\nDeferred Pymt.': 'Residual Receipts / Deferred Pymt.','Annual Debt\nService': 'Annual Debt Service', 'Amount of\nFunds': 'const_l1_fund'})
pf_24['perm_funds_total'] = pf_24.iloc[12, 5]
pf_24['tc_equity'] = pf_24.iloc[13, 5]
pf_24['projectfunds_total'] = pf_24.iloc[14, 5]
pf_24

Table extracted and saved to 24app_permfin.csv


,perm_l1_name,perm_l1_term,const_l1_ir,Residual Receipts / Deferred Pymt.,Annual Debt Service,const_l1_fund,perm_funds_total,tc_equity,projectfunds_total
0,Sonoma County CDC - Fund for Housing,660.0,3.000%,Residual,NaN,"$2,273,440","$4,183,502","$13,647,151","$17,830,653"
1,Federal Home Loan Bank-SF-AHP,660.0,zero,Deferred,NaN,"$800,000","$4,183,502","$13,647,151","$17,830,653"
2,Land Donation from Unrelated Party,NaN,NaN,NaN,NaN,"$1,108,316","$4,183,502","$13,647,151","$17,830,653"
3,GP Equity,NaN,NaN,NaN,NaN,"$1,000","$4,183,502","$13,647,151","$17,830,653"
4,Deferred Developer Fee,NaN,NaN,NaN,NaN,$746,"$4,183,502","$13,647,151","$17,830,653"
5,NaN,NaN,NaN,NaN,NaN,NaN,"$4,183,502","$13,647,151","$17,830,653"
6,NaN,NaN,NaN,NaN,NaN,NaN,"$4,183,502","$13,647,151","$17,830,653"
7,NaN,NaN,NaN,NaN,NaN,NaN,"$4,183,502","$13,647,151","$17,830,653"
8,NaN,NaN,NaN,NaN,NaN,NaN,"$4,183,502","$13,647,151","$17,830,653"
9,NaN,NaN,NaN,NaN,NaN,NaN,"$4,183,502","$13,647,151","$17,830,653"


In [1]:
list(pf_24['perm_l1_name'])

NameError: name 'pf_24' is not defined

In [7]:
# import tabula
# import pandas as pd

# # Define the PDF file path
# pdf_path = "path/to/your/file.pdf"

# # Define the area for the table extraction (if needed)
# area = [50, 50, 400, 500]  # top, left, bottom, right coordinates

# Extract the table using the 'stream' method
# tables_stream = tabula.read_pdf(pdf_path, pages=1, area=area, guess=False, stream=True)
# df_stream = tables_stream[0]

# # Extract the table using the 'lattice' method
# #tables_lattice = tabula.convert_into(app_24, "24app_9%.csv", output_format="csv", pages=[12], lattice=True)
# tables_lattice = tabula.read_pdf(app_24, pages=12, lattice=True)
# df_lattice = tables_lattice[0]
# df_lattice
# #tables_lattice

# # Example post-processing: Rename columns
# df_stream.columns = ["Column1", "Column2", "Column3", "Column4"]
# df_lattice.columns = ["Column1", "Column2", "Column3", "Column4"]

# # Handle missing values (example: fill with empty string)
# df_stream.fillna("", inplace=True)
# df_lattice.fillna("", inplace=True)

# # Display the cleaned DataFrames
# print("Cleaned Stream Method:")
# print(df_stream)

# print("Cleaned Lattice Method:")
# print(df_lattice)


,III. PROJECT FINANCING - SECTION 1: CONSTRUCTION FINANCING


In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive", force_remount=True)
# app4 = "/content/gdrive/MyDrive/Terner/PDFs/17-004.pdf"
# import_and_clean_table(app4)


In [ ]:
# from collections import Counter, defaultdict
# from itertools import combinations
# import pandas as pd
# import numpy as np
# import operator
# import math
# import seaborn as sns
# import itertools
# from sklearn.feature_extraction import DictVectorizer
# from sklearn import preprocessing, tree
# import matplotlib.pyplot as plt
# from sklearn import metrics
# %matplotlib inline
# import zipfile

# import torch

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.optimizers.legacy import Adam
# # Modelling
# from sklearn.model_selection import cross_val_score
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import AdaBoostClassifier
# #https://www.datacamp.com/tutorial/understanding-logistic-regression-python
# from sklearn.ensemble import RandomForestClassifier
# #https://www.datacamp.com/tutorial/random-forests-classifier-python
# from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
# from sklearn.model_selection import RandomizedSearchCV, train_test_split
# from scipy.stats import randint
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# import seaborn as sns

In [ ]:
#-99 blank,-98 for N/A